# Neuron, Astro, Not Cellを選択するGUI

In [2]:
# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
import datetime

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont
from PyQt5.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.utils import *

class Suite2pROICheckGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Suite2pROICheckGUI")
        self.setGeometry(100, 100, 1200, 200)
        self.setupUI_done = False

        self.dict_label    = {} # labelの保管
        self.dict_entry    = {} # entryの保管
        self.dict_button   = {} # buttonの保管
        self.dict_ax       = {} # axの保管
        self.dict_checkbox = {} # checkboxの保管
        self.dict_slider   = {} # sliderの保管
        self.dict_buttongroup = {}
        self.setupButtonUI()
        
    def initializeGUI(self, Fall, list_ROI, cell_selections={}):
        self.Fall_iscell = Fall["iscell"][:, 0]
        self.current_slice = 0
        self.list_ROI = list_ROI
        self.list_ROIsize = [len(roi["xpix"]) for roi in gui.list_ROI]
        self.channel_visibility = {"Green": True, "Red": True}
        self.showAllROIs = True  # Show All ROIsの初期状態をTrueに設定
        self.distAllROIsMoved = [0, 0] # WASDキーで動かしたAll ROIs imageの移動距離
        self.current_opacity = 255
        self.current_opacity_allroi = 50
#         self.cell_selections = cell_selections
        self.Fall_F = Fall["F"]
        self.Fall_Fneu = Fall["Fneu"]
        self.Fall_spks = Fall["spks"]
        self.Fall_stat = Fall["stat"]
        ops_key   = list(Fall["ops"].dtype.fields)
        ops_value = Fall["ops"].tolist()[0][0]
        self.dict_Fall_ops = {k:v for k,v in zip(ops_key, ops_value)}
        # statをcell idをkeyとするdictに変換
        dict_Fall_stat = {cellid:dict(zip(list(Fall["stat"][0][cellid][0].dtype.fields), list(Fall["stat"][0][cellid][0][0]))) for cellid in range(len(Fall["stat"][0]))}
        self.dict_Fall_stat = dict_Fall_stat
        self.xlim = None
        self.roiColors = {}  # 各ROIに割り当てられた色を保持する辞書を追加
        self.contrastValues = {"Green": {'min': 0, 'max': 255}, "Red": {'min': 0, 'max': 255}}
        self.load_eventfile = False
        # 表示するref画像
        self.image_stacks = {
            "Green": np.array([self.dict_Fall_ops["meanImg"]]),
            "Red": self.refimg_R,
        }
        # ROI用の色をランダムに選択(暗すぎない色)
        for index, roi in enumerate(self.list_ROI):
            color = QColor(np.random.randint(100, 255), np.random.randint(100, 255), np.random.randint(100, 255), 50)
            self.roiColors[index] = color
        self.dict_showROI = {0:"all", 1:"astro", 2:"neuron", 3:"noise", 4:"none"}
        # Tableの列名と列番号のdict
        self.dict_celltypes = {"Astrocyte": 1, "Neuron": 2, "Not Cell": 3, "Check": 4, "Tracking": 5, "Memo": 6}
        if hasattr(self, "tableWidget"):
            self.tableWidget.clearSelection() # テーブルの選択初期化
            self.tableWidget.clear()  # テーブルの内容をクリア
            self.tableWidget.setRowCount(0)  # 行数を0にリセット
            self.tableWidget.deleteLater() # テーブルウィジェット削除
            self.setupTableWidget()  # テーブルウィジェットを再設定
            self.mainLayout.addWidget(self.tableWidget, 0, 4, 1, 2) # テーブルウィジェットを再配置
            self.tableWidget.setMinimumWidth(550)
        # UIの初期設定
        if not self.setupUI_done:
            self.setGeometry(100, 100, 1800, 1000)
            self.setupUI()
            self.setupUI_done = True

        self.createROIPixmap()  # ROIのピクセルマップを作成
        self.updateImageDisplay()
        for ax in self.dict_ax.values():
            ax.clear()
        self.plotMeanTrace()
        self.countAndshowROINumber()
        
    # 画像をint型に変換 型は指定可能
    def convertImageToINT(self, im, dtype="uint8"):
        im = im.astype("float")
        im -= np.min(im)
        im /= np.max(im)
        im *= 255
        im = im.astype(dtype)
        return im
    # ファイルの読み込み
    def loadFallImageMaskROIs(self, path_Fall, path_im_cellpose_G, path_mask_cellpose_R):
        Fall = loadmat(path_Fall)
        Fall_stat = Fall["stat"]
        Fall_iscell = Fall["iscell"][:,0]

        list_ROI = []

        for cellid in range(len(Fall_iscell)):
            dict_Fall_stat_cell = {key:value for key, value in zip(Fall_stat[0][cellid][0].dtype.fields, Fall_stat[0][cellid][0][0])}
            xpix = dict_Fall_stat_cell["xpix"][0]
            ypix = dict_Fall_stat_cell["ypix"][0]
            center = (int(xpix.mean()), int(ypix.mean())) # ROIの中心
            list_ROI.append({"xpix": xpix, "ypix": ypix, "center": center})
        
        if path_im_cellpose_G:
            im_cellpose_G = tifffile.imread(path_im_cellpose_G)
            im_cellpose_G = self.convertImageToINT(im_cellpose_G)
        else:
            im_cellpose_G = np.zeros((512, 512))
            
        if path_mask_cellpose_R:
            mask_cellpose_R = np.load(path_mask_cellpose_R, allow_pickle=True)

            # Redのマスク画像をim_cellpose_Rとする
            im_cellpose_R = mask_cellpose_R.tolist()["outlines"]
            im_cellpose_R[im_cellpose_R > 0] = 255
            im_cellpose_R = im_cellpose_R.astype("uint8")
        else:
            im_cellpose_R = None
        
        return Fall, im_cellpose_G, im_cellpose_R, list_ROI
        
    # 指定したファイルパスを読み込みして初期化
    def loadFilePathsandInitialize(self):
        self.initialize_gui = False
        print("Files Loading...")
        # Fall
        path_Fall = self.dict_entry["path_fall"].text()
        # cellposeで取得した細胞のマスク画像
        path_mask_cellpose_R = self.dict_entry["path_cellpose"].text()
        # 細胞選択用の参照画像
        path_im_cellpose_G = self.dict_entry["path_reftif"].text()
        # プロジェクトディレクトリ, 個体番号, 日付, plane番号
        self.dir_project = path_Fall.rsplit("/", 1)[0]
        self.mouse = self.dir_project.split("/")[-2]
        self.date = self.dir_project.split("/")[-1]

        Fall, im_cellpose_G, im_cellpose_R, list_ROI = self.loadFallImageMaskROIs(path_Fall, path_im_cellpose_G, path_mask_cellpose_R)
        self.refimg = np.array([im_cellpose_G])
        self.refimg_R = np.array([im_cellpose_R])
        
        # 初期化
        print("GUI Initializing...")
        self.initializeGUI(Fall, list_ROI)
        self.initialize_gui = True
        
    # ボタンのみのUIセットアップ
    def setupButtonUI(self):
        self.mainLayout = QGridLayout()
        
        setupLayout = QVBoxLayout()
        # ファイルパスとbrowseボタン
        layout_path_fall = self.makeLoadFileWidgetLayout(label="Fall mat file path", key="path_fall", fileFilter="mat Files (*.mat);;All Files (*)")
        layout_path_reftif = self.makeLoadFileWidgetLayout(label="Reference image tiff file path (optional)", key="path_reftif", fileFilter="tiff Files (*.tif *.tiff);;All Files (*)")
        layout_path_cellpose = self.makeLoadFileWidgetLayout(label="Cellpose mask npy file path (optional)", key="path_cellpose", fileFilter="npy Files (*.npy);;All Files (*)")
        
        self.buttonLayout_setup = QHBoxLayout()
        # Load
        self.dict_button["loadFile"] = QPushButton("Load files")
        self.dict_button["loadFile"].clicked.connect(self.loadFilePathsandInitialize)
        self.buttonLayout_setup.addWidget(self.dict_button["loadFile"])
        # Exitボタンの設定
        self.dict_button["exit"] = QPushButton("Exit")
        self.dict_button["exit"].clicked.connect(self.close)
        self.buttonLayout_setup.addWidget(self.dict_button["exit"])
        
        setupLayout.addLayout(layout_path_fall)
        setupLayout.addLayout(layout_path_reftif)
        setupLayout.addLayout(layout_path_cellpose)
        setupLayout.addLayout(self.buttonLayout_setup)
        self.mainLayout.addLayout(setupLayout, 4, 0, 1, 6)
        
        widget = QWidget()
        widget.setLayout(self.mainLayout)
        self.setCentralWidget(widget)

    # 起動時のUI初期化
    def setupUI(self):
        # 画像表示用の設定
        self.scene = QGraphicsScene()
        self.view = QGraphicsView(self.scene)
        self.view.setMinimumHeight(550)
        self.view.setMinimumWidth(550)

        labelLayout_roitrace = QVBoxLayout()
        # ROIのsize, radius, aspect_ratio, compact, footprint
        labelLayout_roi = QHBoxLayout()
        for label_key in (["npix", "radius", "aspect_ratio", "compact", "footprint"]):
            self.dict_label[f"ROI_{label_key}"] = QLabel(f"{label_key}: ")
            self.dict_label[f"ROI_{label_key}"].setAlignment(Qt.AlignLeft)  # テキストを左揃えに
            labelLayout_roi.addWidget(self.dict_label[f"ROI_{label_key}"])
        # 選択したROI traceのskew(歪度), std
        labelLayout_trace = QHBoxLayout()
        for label_key in (["skew", "std"]):
            self.dict_label[f"ROI_{label_key}"] = QLabel(f"{label_key}: ")
            self.dict_label[f"ROI_{label_key}"].setAlignment(Qt.AlignLeft)  # テキストを左揃えに
            labelLayout_trace.addWidget(self.dict_label[f"ROI_{label_key}"])
        labelLayout_roitrace.addLayout(labelLayout_roi)
        labelLayout_roitrace.addLayout(labelLayout_trace)
    
        
        buttonLayout_roidisp = QVBoxLayout()
        # ROIの表示切り替え All, Cell, Not Cell
        buttonLayout_roidisp_radiobutton = QHBoxLayout()
        self.dict_buttongroup["roicheck"] = QButtonGroup(self)
        self.dict_buttongroup["roicheck"].setExclusive(True)  # 同一グループ内で1つだけ選択可能にする
        for i, label in enumerate(["All ROI", "Astrocyte", "Neuron", "Not Cell", "None"]):
            radioButton = QRadioButton(label)
            if i == 0:
                radioButton.setChecked(True)
            buttonLayout_roidisp_radiobutton.addWidget(radioButton)
            self.dict_buttongroup["roicheck"].addButton(radioButton, i)
            # ボタンが選択されたときのイベントハンドラを設定
            radioButton.toggled.connect(lambda checked, row=i, choice=label: self.onRadioButtonshowROIToggled(checked, row, choice))
        buttonLayout_roidisp.addLayout(buttonLayout_roidisp_radiobutton)
        
        # 背景画像の切り替え meanImg, meanImgE, max_proj, Vcorr, RefImg
        buttonLayout_roidisp_refimg = QHBoxLayout()
        self.dict_buttongroup["refimg"] = QButtonGroup(self)
        self.dict_buttongroup["refimg"].setExclusive(True)  # 同一グループ内で1つだけ選択可能にする
        for i, label in enumerate(["meanImg", "meanImgE", "max_proj", "Vcorr", "RefImg"]):
            radioButton = QRadioButton(label)
            if i == 0:
                radioButton.setChecked(True)
            buttonLayout_roidisp_refimg.addWidget(radioButton)
            self.dict_buttongroup["refimg"].addButton(radioButton, i)
            # ボタンが選択されたときのイベントハンドラを設定
            radioButton.toggled.connect(lambda checked, row=i, choice=label: self.onRadioButtonshowImageToggled(checked, row, choice))
        buttonLayout_roidisp.addLayout(buttonLayout_roidisp_refimg)
        
        buttonLayout_roidisp_view = QHBoxLayout()
        # 画面クリック時 Astrocyte, Neuron, Not Cell, Check, TrackingのROIをスキップするか
        for celltype in ["Astrocyte", "Neuron", "NotCell", "Check", "Tracking"]:
            self.dict_checkbox[f"skip{celltype}"] = QCheckBox(f"Skip '{celltype}' ROI")
            self.dict_checkbox[f"skip{celltype}"].setChecked(False)  
            buttonLayout_roidisp_view.addWidget(self.dict_checkbox[f"skip{celltype}"])

        # Move ROIs Mask Imageチェックボックスの設定
        self.dict_checkbox["moveROIImage"] = QCheckBox("Move ROI Mask Image")
        self.dict_checkbox["moveROIImage"].setChecked(False)  
        buttonLayout_roidisp_view.addWidget(self.dict_checkbox["moveROIImage"])
        buttonLayout_roidisp.addLayout(buttonLayout_roidisp_view)

        # スライダーの配置
        sliderLayout_ALL = QVBoxLayout()
        
        sliderLayout_contrast = QHBoxLayout()
        # コントラストスライダーとチェックボックスの設定
        for i, channel in enumerate(["Green", "Red"]):
            sliderLayout = self.makeContrastSlidersLayout(channel)
            sliderLayout_contrast.addLayout(sliderLayout)
            
        sliderLayout_ALL.addLayout(sliderLayout_contrast)
        
        # 全てのROIの透明度スライダーの設定（オプション）
        sliderLayout_opecity = QHBoxLayout()
        for (key, label, default_value, func_) in zip(['opacity_allroi', 'opacity'],['Opacity of All ROI', 'Opacity of Selected ROI'],
                                                         [50, 255],[self.changeAllROIOpacity, self.changeSelectedROIOpacity]):
            sliderLayout = QVBoxLayout()
            self.dict_slider[key] = QSlider(Qt.Horizontal)
            self.dict_slider[key].setMinimum(0)
            self.dict_slider[key].setMaximum(255)
            self.dict_slider[key].setMaximumHeight(10)
            self.dict_slider[key].setValue(default_value)
            self.dict_slider[key].valueChanged.connect(func_)
            self.dict_label[key] = QLabel(label)
            sliderLayout.addWidget(self.dict_label[key])
            sliderLayout.addWidget(self.dict_slider[key])
            sliderLayout_opecity.addLayout(sliderLayout)
            
        sliderLayout_ALL.addLayout(sliderLayout_opecity)
        
        

        # テーブルウィジェットの設定
        self.setupTableWidget()
        # サイズ指定
        self.tableWidget.setMinimumWidth(550)
                                         
        labelbuttonLayout_roicheck = QVBoxLayout()
        # ROI number label
        self.dict_label["ROInumber"] = QLabel("Astrocyte: 0, Neuron: 0, Not Cell: 0, All: 0")  # ピクセル数を表示するラベルを作成
        self.dict_label["ROInumber"].setAlignment(Qt.AlignLeft)  # テキストを左揃えに
        labelbuttonLayout_roicheck.addWidget(self.dict_label["ROInumber"])  # レイアウトにラベルを追加
        
        # All ROI set button, QHBoxLayoutにまとめる
        buttonLayout_roicheck = QHBoxLayout()
        for key, label, func_ in zip(["setAstrocyte", "setNeuron", "setNoise", "setCheck", "clearCheck"],
                                      ["Astrocyte", "Neuron", "Not Cell", "Check", "Clear Check"],
                                      [lambda: self.setAllROIClass("Astrocyte"), 
                                       lambda: self.setAllROIClass("Neuron"), 
                                       lambda: self.setAllROIClass("Not Cell"), 
                                       self.setAllROICheckBox, 
                                       self.clearAllROICheckBox]):
            self.dict_button[key] = QPushButton(label)
            self.dict_button[key].clicked.connect(func_)
            buttonLayout_roicheck.addWidget(self.dict_button[key])
        
        labelbuttonLayout_roicheck.addLayout(buttonLayout_roicheck)
        
        # ROICheck save, loadボタン
        buttonLayout_roiIO = QHBoxLayout()
        self.dict_button["saveROICheck"] = QPushButton("Save ROICheck")
        self.dict_button["saveROICheck"].clicked.connect(self.saveROICheck)
        buttonLayout_roiIO.addWidget(self.dict_button["saveROICheck"])
        self.dict_button["loadROICheck"] = QPushButton("Load ROICheck")
        self.dict_button["loadROICheck"].clicked.connect(self.loadROICheck)
        buttonLayout_roiIO.addWidget(self.dict_button["loadROICheck"])
        self.dict_button["loadROIMatch"] = QPushButton("Load ROI Match")
        self.dict_button["loadROIMatch"].clicked.connect(self.loadROIMatch)
        buttonLayout_roiIO.addWidget(self.dict_button["loadROIMatch"])
        
        labelbuttonLayout_roicheck.addLayout(buttonLayout_roiIO)
        
        # Cell Filter 2x3 boxlayout
        entryLayout_filterroi = QVBoxLayout()
        for j, (parames, thresholds_default) in enumerate(zip([["npix", "radius", "aspect_ratio"],["compact", "skew", "std"]], 
                                                              [["(50, 200)", "(3, 12)", "(0, 1.5)"],["(0, 1.5)", "(1, 100)", "(0, 100)"]])):
            entrylayout = QHBoxLayout()
            for i, (param, threshold_default) in enumerate(zip(parames, thresholds_default)):
                entrylayou = QVBoxLayout()
                self.dict_label[f"threshold_{param}"] = QLabel(f"{param}")
                entrylayou.addWidget(self.dict_label[f"threshold_{param}"])
                self.dict_entry[f"threshold_{param}"] = QLineEdit()
                self.dict_entry[f"threshold_{param}"].setFixedWidth(100)
                self.dict_entry[f"threshold_{param}"].setText(threshold_default)
                entrylayou.addWidget(self.dict_entry[f"threshold_{param}"])   
                entrylayout.addLayout(entrylayou)
            entryLayout_filterroi.addLayout(entrylayout)
            
        buttonlabelLayout_filterroi = QVBoxLayout()
        
        self.dict_label[f"threshold_params"] = QLabel("<- thresholds (min, max)")
        buttonlabelLayout_filterroi.addWidget(self.dict_label[f"threshold_params"])
        self.dict_button["filterROI"] = QPushButton("Filter ROI")
        self.dict_button["filterROI"].clicked.connect(self.filterROI)
        buttonlabelLayout_filterroi.addWidget(self.dict_button["filterROI"])

        # プロットウィジェットの設定
        self.plotWidget = QWidget()
        plotLayout = QVBoxLayout(self.plotWidget)
        self.figure = Figure()
        self.canvas = FigureCanvas(self.figure)
        plotLayout.addWidget(self.canvas)
        self.dict_ax["top"] = self.figure.add_subplot(3, 1, 1)
        self.dict_ax["middle"] = self.figure.add_subplot(3, 1, 2)
        self.dict_ax["bottom"] = self.figure.add_subplot(3, 1, 3)
        self.figure.subplots_adjust(top=0.95, bottom=0.1, right=0.9, left=0.1, hspace=0.4)
        # スクロールイベントをキャンバスに接続
        self.canvas.mpl_connect('scroll_event', self.on_scroll)
         # マウスイベントの接続
        self.canvas.mpl_connect('button_press_event', self.on_press)
        self.canvas.mpl_connect('motion_notify_event', self.on_motion)
        self.canvas.mpl_connect('button_release_event', self.on_release)
        self.is_dragging = False

        labelentrycheckboxLayout_eventalign = QHBoxLayout()
        # EventFileにalignしたtraceのプロットコンフィグ
        labelentryLayout_eventalign = QVBoxLayout()
        self.dict_label["EventFileAlignedTrace"] = QLabel("plot range from Event start (pre, post; frames)")
        labelentryLayout_eventalign.addWidget(self.dict_label["EventFileAlignedTrace"])
        self.dict_entry["EventFileAlignedTrace"] = QLineEdit()
        self.dict_entry["EventFileAlignedTrace"].setText("(50, 50)")
        labelentryLayout_eventalign.addWidget(self.dict_entry["EventFileAlignedTrace"])
        # プロットに重ねるEventFile npyファイルの読み込みボタン
        buttonLayout_eventalign = QHBoxLayout()
        self.dict_button["loadEventFile"] = QPushButton("Load EventFile npy file")
        self.dict_button["loadEventFile"].clicked.connect(self.loadEventFile)
        buttonLayout_eventalign.addWidget(self.dict_button["loadEventFile"])
        self.dict_button["clearEventFile"] = QPushButton("Clear")
        self.dict_button["clearEventFile"].clicked.connect(self.clearEventFile)
        buttonLayout_eventalign.addWidget(self.dict_button["clearEventFile"])
        
        labelentryLayout_eventalign.addLayout(buttonLayout_eventalign)
        
        # zoomの最大値
        labelentryLayout_zoomrange = QVBoxLayout()
        self.dict_label["minPlotRange"] = QLabel("Minimum plot range (sec)")
        labelentryLayout_zoomrange.addWidget(self.dict_label["minPlotRange"])
        self.dict_entry["minPlotRange"] = QLineEdit()
        self.dict_entry["minPlotRange"].setText("30")
        labelentryLayout_zoomrange.addWidget(self.dict_entry["minPlotRange"])
        
        checkboxLayout_eventalign = QVBoxLayout()
        # 読み込んだEventFileのtraceをプロットするか
        self.dict_checkbox["plotEventFileTrace"] = QCheckBox("plot EventFile trace")
        self.dict_checkbox["plotEventFileTrace"].setChecked(True)  # デフォルトでチェックされている
        checkboxLayout_eventalign.addWidget(self.dict_checkbox["plotEventFileTrace"])
        
        labelentrycheckboxLayout_eventalign.addLayout(labelentryLayout_eventalign)
        labelentrycheckboxLayout_eventalign.addLayout(labelentryLayout_zoomrange)
        labelentrycheckboxLayout_eventalign.addLayout(checkboxLayout_eventalign)
        
        # Helpボタンの設定
        self.dict_button["help"] = QPushButton("help")
        self.dict_button["help"].clicked.connect(self.showHelp)
        
        # Widget, Layoutの配置
        # 左
        self.mainLayout.addWidget(self.plotWidget, 0, 0, 3, 2)
        self.mainLayout.addLayout(labelentrycheckboxLayout_eventalign, 3, 0, 1, 2)
        # 中
        self.mainLayout.addWidget(self.view, 0, 2, 1, 2)
        self.mainLayout.addLayout(labelLayout_roitrace, 1, 2, 1, 2)
        self.mainLayout.addLayout(buttonLayout_roidisp, 2, 2, 1, 2)
        self.mainLayout.addLayout(sliderLayout_ALL, 3, 2, 1, 2)
        # 右
        self.mainLayout.addWidget(self.tableWidget, 0, 4, 1, 2)
        self.mainLayout.addLayout(labelbuttonLayout_roicheck, 1, 4, 2, 2)
        self.mainLayout.addLayout(entryLayout_filterroi, 3, 4, 1, 1)
        self.mainLayout.addLayout(buttonlabelLayout_filterroi, 3, 5, 1, 1)
        # すでに作成したBoxlayoutに追加
        self.buttonLayout_setup.addWidget(self.dict_button["help"])
        
        widget = QWidget()
        widget.setLayout(self.mainLayout)
        self.setCentralWidget(widget)
        
        # self.viewのmousePressEventをオーバーライドするための設定
        self.view.mousePressEvent = self.viewMousePressEvent
        
    # 読み込むファイルを選択するためのウィジェット
    def makeLoadFileWidgetLayout(self, label="", key="", fileFilter=""):
        hboxlayout = QHBoxLayout() # entry, button
        vboxlayout = QVBoxLayout() # label, entry. button
        
        self.dict_label[key] = QLabel(label)
        self.dict_entry[key] = QLineEdit()
        self.dict_button[f"browse_{key}"] = QPushButton("Browse")
        self.dict_button[f"browse_{key}"].clicked.connect(lambda: self.openFileDialog(fileFilter, self.dict_entry[key]))
        hboxlayout.addWidget(self.dict_entry[key])
        hboxlayout.addWidget(self.dict_button[f"browse_{key}"])
        vboxlayout.addWidget(self.dict_label[key])
        vboxlayout.addLayout(hboxlayout)
        return vboxlayout
        
    def makeContrastSlidersLayout(self, channel):
        # 各チャンネルのコントラストスライダーとチェックボックスの設定
        sliderLayout = QVBoxLayout()

        # チェックボックスの設定
        self.dict_checkbox[f"show_{channel}"] = QCheckBox(f"Show {channel}")
        self.dict_checkbox[f"show_{channel}"].setChecked(True)
        self.dict_checkbox[f"show_{channel}"].stateChanged.connect(lambda state, ch=channel: self.toggleChannelVisibility(ch, state))
        sliderLayout.addWidget(self.dict_checkbox[f"show_{channel}"])

        # Min, Max Valueスライダーの設定
        for m, default_value in zip(["min", "max"], [0, 255]):
            self.dict_slider[f"{m}Value_{channel}"] = QSlider(Qt.Horizontal)
            self.dict_slider[f"{m}Value_{channel}"].setMinimum(0)
            self.dict_slider[f"{m}Value_{channel}"].setMaximum(255)
            self.dict_slider[f"{m}Value_{channel}"].setMaximumHeight(10)
            self.dict_slider[f"{m}Value_{channel}"].setValue(default_value)
            # ラムダ関数のデフォルト引数を使ってループ変数の値をキャプチャ, これがないと両方のスライダーがmaxとなる
            self.dict_slider[f"{m}Value_{channel}"].valueChanged.connect(lambda value, ch=channel, m=m: self.adjustContrast(ch, m, value))
            self.dict_label[f"{m}Value_{channel}"] = QLabel(f"{m} Value ({channel})")
            sliderLayout.addWidget(self.dict_label[f"{m}Value_{channel}"])
            sliderLayout.addWidget(self.dict_slider[f"{m}Value_{channel}"])

        return sliderLayout
        
    def openFileDialog(self, fileFilter, entry):
        options = QFileDialog.Options()
        filePath, _ = QFileDialog.getOpenFileName(self, "Open File", "", fileFilter, options=options)
        if filePath:
            entry.setText(filePath)
            
    """
    Table Widget Function
    """
        
    def setupTableWidget(self):
        self.tableWidget = QTableWidget()
        self.tableWidget.setRowCount(len(self.Fall_iscell))
        self.tableWidget.setColumnCount(len(self.dict_celltypes)+1)
        self.tableWidget.setHorizontalHeaderLabels(["Cell ID"] + list(self.dict_celltypes.keys()))
        self.buttonGroups = []
        self.currently_selected_row = None  # 追加: 現在選択されている行を追跡

        for i, iscell in enumerate(self.Fall_iscell):
            cellItem = QTableWidgetItem(f"Cell {i}")
            cellItem.setFlags(cellItem.flags() & ~Qt.ItemIsEditable)
            self.tableWidget.setItem(i, 0, cellItem)
            buttonGroup = QButtonGroup(self.tableWidget)
            buttonGroup.setExclusive(True)
            for label, j in self.dict_celltypes.items():
                # Astrocyte, Neuron. Not Cell
                if label not in ["Check", "Tracking", "Memo"]:
                    radioButton = QRadioButton(label)
                    buttonGroup.addButton(radioButton, j)
                    self.tableWidget.setCellWidget(i, j, radioButton)
                    radioButton.toggled.connect(lambda checked, row=i, choice=label: self.onRadioButtonToggled(checked, row, choice))
                    if j == 1: # 2列目チェック
                        radioButton.setChecked(True)
            self.buttonGroups.append(buttonGroup)
            # Check. Tracking
            for label in ["Check", "Tracking"]:
                checkBox = QTableWidgetItem()
                checkBox.setCheckState(Qt.Unchecked)
                self.tableWidget.setItem(i, self.dict_celltypes[label], checkBox)
            # Memo
            memoItem = QTableWidgetItem()
            self.tableWidget.setItem(i, self.dict_celltypes["Memo"], memoItem)
            
            # セルの横幅指定
            for i in range(self.tableWidget.columnCount()):
                self.tableWidget.setColumnWidth(i, 75)
            self.tableWidget.setColumnWidth(self.tableWidget.columnCount(), 200)
            
        self.tableWidget.selectionModel().selectionChanged.connect(self.onSelectionChanged)
        
    # キーイベントのカスタム
    def keyPressEvent(self, event):
        # ROIを選択しているとき
        if not self.currently_selected_row is None:
            currentRow = self.currently_selected_row # 現在選択している行番号
            currentRows = self.tableWidget.selectionModel().selectedRows()# 現在選択している行番号(複数)
            currentColumn = self.tableWidget.currentColumn() # 現在選択している列番号
            currentCelltype = self.buttonGroups[currentRow].checkedId() # 現在の行で選択しているcelltypeの列番号
                    
            if event.key() == Qt.Key_Z:
                self.checkRadioButton(self.currently_selected_row, "Astrocyte")
            elif event.key() == Qt.Key_X:
                self.checkRadioButton(self.currently_selected_row, "Neuron")
            elif event.key() == Qt.Key_C:
                self.checkRadioButton(self.currently_selected_row, "Not Cell")
            elif event.key() == Qt.Key_V:
                self.toggleCheckState(self.currently_selected_row)
            elif event.key() == Qt.Key_B:
                self.toggleTrackingState(self.currently_selected_row)
            elif event.key() == Qt.Key_G:
                current_state = self.dict_checkbox["show_Green"].isChecked()
                self.dict_checkbox["show_Green"].setChecked(not current_state)
            # 行移動, U,I,O:上に移動 J,K,L:下に移動, (I,K:"Check"ありはスキップ, O,L:"Check"なしはスキップ, U,J:同じcelltypeのみ)
            elif event.key() == Qt.Key_J:
                while True:
                    currentRow += 1
                    if currentRow >= self.tableWidget.rowCount():
                        break  # テーブルの末尾に到達したら停止
                    if self.buttonGroups[currentRow].checkedId() == currentCelltype:
                        self.tableWidget.setCurrentCell(currentRow, currentColumn)
                        return  # ここで処理を終了
            elif event.key() == Qt.Key_U:
                while True:
                    currentRow -= 1
                    if currentRow < 0:
                        break  # テーブルの末尾に到達したら停止
                    if self.buttonGroups[currentRow].checkedId() == currentCelltype:
                        self.tableWidget.setCurrentCell(currentRow, currentColumn)
                        return  # ここで処理を終了
            elif event.key() == Qt.Key_K:  # 'K'キーが押された時
                while True:
                    currentRow += 1
                    if currentRow >= self.tableWidget.rowCount():
                        break  # テーブルの末尾に到達したら停止
                    if self.tableWidget.item(currentRow, 4).checkState() != Qt.Checked:
                        self.tableWidget.setCurrentCell(currentRow, currentColumn)
                        return  # ここで処理を終了
            elif event.key() == Qt.Key_I:
                while True:
                    currentRow -= 1
                    if currentRow < 0:
                        break  # テーブルの先頭に到達したら停止
                    if self.tableWidget.item(currentRow, 4).checkState() != Qt.Checked:
                        self.tableWidget.setCurrentCell(currentRow, currentColumn)
                        return  # ここで処理を終了
            elif event.key() == Qt.Key_L:
                while True:
                    currentRow += 1
                    if currentRow >= self.tableWidget.rowCount():
                        break  # テーブルの末尾に到達したら停止
                    if self.tableWidget.item(currentRow, 4).checkState() == Qt.Checked:
                        self.tableWidget.setCurrentCell(currentRow, currentColumn)
                        return  # ここで処理を終了
            elif event.key() == Qt.Key_O:
                while True:
                    currentRow -= 1
                    if currentRow < 0:
                        break  # テーブルの先頭に到達したら停止
                    if self.tableWidget.item(currentRow, 4).checkState() == Qt.Checked:
                        self.tableWidget.setCurrentCell(currentRow, currentColumn)
                        return  # ここで処理を終了
                
        # dict_checkbox["moveROIImage"]がチェックされているか確認
        if self.dict_checkbox["moveROIImage"].isChecked():
            # 移動量を設定
            moveDistance = 1  # 1ピクセル単位で移動

            # 矢印キーに応じて位置を更新
            if event.key() == Qt.Key_D:
                self.distAllROIsMoved[0] += moveDistance
            elif event.key() == Qt.Key_A:
                self.distAllROIsMoved[0] -= moveDistance
            elif event.key() == Qt.Key_W:
                self.distAllROIsMoved[1] -= moveDistance
            elif event.key() == Qt.Key_S:
                self.distAllROIsMoved[1] += moveDistance
                
        # ROIpixmapの更新
        id_showROI = self.dict_showROI[self.dict_buttongroup["roicheck"].checkedId()]
        self.createROIPixmap(id_showROI)

    def checkRadioButton(self, row, label):
        labels = ["Astrocyte", "Neuron", "Not Cell"]
        for j, lbl in enumerate(labels, start=1):
            radioButton = self.tableWidget.cellWidget(row, j)
            checkItem = self.tableWidget.item(row, self.dict_celltypes["Check"])  # "Check"列のインデックスは4
            isChecked = checkItem.checkState() == Qt.Checked
            if (lbl == label) and not isChecked: # Checkがつけられている場合はパス
                radioButton.setChecked(True)
    # Check列のチェック切り替え
    def toggleCheckState(self, row):
        checkBoxItem = self.tableWidget.item(row, self.dict_celltypes["Check"])
        if checkBoxItem.checkState() == Qt.Checked:
            checkBoxItem.setCheckState(Qt.Unchecked)
        else:
            checkBoxItem.setCheckState(Qt.Checked)
    # Tracking列のチェック切り替え
    def toggleTrackingState(self, row):
        checkBoxItem = self.tableWidget.item(row, self.dict_celltypes["Tracking"])
        if checkBoxItem.checkState() == Qt.Checked:
            checkBoxItem.setCheckState(Qt.Unchecked)
        else:
            checkBoxItem.setCheckState(Qt.Checked)
    # Tableのradiobuttonの関数
    def onRadioButtonToggled(self, checked, row, choice):
        """RadioButtonの選択状態が変更されたときに呼び出されるスロット"""
        if checked: # ボタンが選択された場合
            if self.initialize_gui:  
                self.countAndshowROINumber()
    # ROIの表示のradiobutton関数  
    def onRadioButtonshowROIToggled(self, checked, row, choice):
        if checked: # ボタンが選択された場合
            self.createROIPixmap(self.dict_showROI[row])
    # reference画像表示のradiobutton関数  
    def onRadioButtonshowImageToggled(self, checked, row, choice):
        if checked: # ボタンが選択された場合
            key_img = self.dict_buttongroup["refimg"].checkedButton().text()
            if key_img == "RefImg": # optionalで指定したReference Image
                self.image_stacks["Green"] = self.refimg
            else:
                # max_proj, Vcorrはxyサイズが異なるので注意
                if key_img in ["max_proj", "Vcorr"]:
                    im = np.zeros((512, 512)).astype(self.dict_Fall_ops[key_img].dtype)
                    im[self.dict_Fall_ops["yrange"][0][0]:self.dict_Fall_ops["yrange"][0][1], self.dict_Fall_ops["xrange"][0][0]:self.dict_Fall_ops["xrange"][0][1]] = self.dict_Fall_ops[key_img]
                else:
                    im = self.dict_Fall_ops[key_img]
                im = self.convertImageToINT(im, dtype="uint16") # intに変換
                self.image_stacks["Green"] = np.array([im])
            self.updateImageDisplay()
    # 別のROIを選択したとき
    def onSelectionChanged(self, selected, deselected):
        indexes = selected.indexes()
        if indexes:
            self.currently_selected_row = indexes[0].row()  # 選択された行を更新
            self.highlightROI(self.currently_selected_row)
            self.updateSelectedCellPlot(self.currently_selected_row)
            self.displayROIstat(self.currently_selected_row)
            
       
    # Astrocyte, Neuron, Not Cellの数をカウントして表示
    def countAndshowROINumber(self):
        num_all = self.tableWidget.rowCount()
        dict_celltype_count = {celltype: 0 for celltype in self.dict_celltypes if celltype not in ["Check", "Tracking", "Memo"]}
        for row in range(num_all):
            for celltype in dict_celltype_count:
                if self.buttonGroups[row].checkedButton().text() == celltype:
                    dict_celltype_count[celltype] += 1
                    break
        text = ", ".join([f"{celltype}: {count}" for celltype, count in dict_celltype_count.items()] + [f"All: {num_all}"])
        self.dict_label["ROInumber"].setText(text)
        
            
    """
    Image, ROI select Function
    """
            
    def viewMousePressEvent(self, event):
        # クリックされた座標をシーンの座標系に変換
        scenePos = self.view.mapToScene(event.pos())
        # 画像の座標系に合わせる
        imgX, imgY = int(scenePos.x()), int(scenePos.y())

        # 最も近いROIを見つける
        closestROI, minDist = None, float('inf')
        for cellid, roi in enumerate(self.list_ROI):
            continue_ = False
            # Astrocyte, Neuron, Not Cellのスキップ
            for i, celltype in enumerate(["Astrocyte", "Neuron", "NotCell"]):
                checkItem = self.tableWidget.cellWidget(cellid, i+1)
                isChecked = checkItem.isChecked()
                if (isChecked) and (self.dict_checkbox[f"skip{celltype}"].isChecked()):
                    continue_ = True
            # Check, Trackingのスキップ
            for i, celltype in enumerate(["Check", "Tracking"]):
                if (self.tableWidget.item(cellid, self.dict_celltypes["Check"]).checkState() == Qt.Checked) and (self.dict_checkbox["skipCheck"].isChecked()):
                    continue_ = True
                    
            # スキップ
            if continue_:
                continue
            centerX, centerY = roi['center']
            # All ROIs imageを動かした分補正
            centerX += self.distAllROIsMoved[0]
            centerY += self.distAllROIsMoved[1]
            dist = (centerX - imgX) ** 2 + (centerY - imgY) ** 2
            if dist < minDist:
                closestROI, minDist = cellid, dist

        if closestROI is not None:
            # テーブルで対応するROIを選択
            self.tableWidget.selectRow(closestROI)
            self.highlightROI(closestROI)
            
    def createROIPixmap(self, showROI="all"):
        num_all = self.tableWidget.rowCount()
        list_celltype = [self.buttonGroups[row].checkedId() for row in range(num_all)]
        # 既存のROIピクセルマップアイテムがあれば削除
        if hasattr(self, "roiPixmapItem"):
            self.scene.removeItem(self.roiPixmapItem)
            self.roiPixmapItem = None
        # ROIを描画するためのピクセルマップを作成
        pixmap = QPixmap(self.image_stacks["Green"].shape[2], self.image_stacks["Green"].shape[1])
        pixmap.fill(Qt.transparent)  # 透明で初期化
        painter = QPainter(pixmap)
        for index, roi in enumerate(self.list_ROI):
            xpix = roi["xpix"] + self.distAllROIsMoved[0] # 補正した分を反映
            ypix = roi["ypix"] + self.distAllROIsMoved[1]
            if self.tableWidget.item(index, self.dict_celltypes["Check"]).checkState() == Qt.Checked: # "Check"した細胞は赤で表示
                color = QColor(255, 0, 0, self.current_opacity_allroi)
            else:
                color = self.roiColors[index]
                color.setAlpha(self.current_opacity_allroi) # 透明度の更新
            pen = QPen(color, 1)
            painter.setPen(pen)
            for x, y in zip(xpix, ypix):
                if showROI=="all":
                    painter.drawPoint(x, y)
                elif (showROI=="astro") and (list_celltype[index]==1): # Astrocyteのみ表示
                    painter.drawPoint(x, y)
                elif (showROI=="neuron") and (list_celltype[index]==2): # Neuronのみ表示
                    painter.drawPoint(x, y)
                elif (showROI=="noise") and (list_celltype[index]==3): # Not Cellのみ表示する
                    painter.drawPoint(x, y)
        painter.end()
        self.roiPixmap = pixmap
        self.roiPixmapItem = QGraphicsPixmapItem(self.roiPixmap)
        self.scene.addItem(self.roiPixmapItem)
            
    def highlightROI(self, selected_row=None):
        if hasattr(self, 'highlightedROIPixmapItem') and self.highlightedROIPixmapItem:
            self.scene.removeItem(self.highlightedROIPixmapItem)
            self.highlightedROIPixmapItem = None

        if selected_row is not None:
            # 透明度の初期化
            for color in self.roiColors.values():
                color.setAlpha(self.current_opacity_allroi)
            # 選択されたROIの色を取得、alpha値を255に設定して不透明にする
            color = self.roiColors[selected_row]
            color.setAlpha(255)  # ハイライト時は不透明に

            # 選択されたROIのピクセルマップを作成
            pixmap = QPixmap(self.image_stacks["Green"].shape[2], self.image_stacks["Green"].shape[1])
            pixmap.fill(Qt.transparent)  # 透明で初期化
            painter = QPainter(pixmap)
            pen = QPen(color, 1)  # 保存された色でハイライト
            painter.setPen(pen)
            roi = self.list_ROI[selected_row]
            for x, y in zip(roi['xpix'], roi['ypix']):
                painter.drawPoint(x + self.distAllROIsMoved[0], y + self.distAllROIsMoved[1])
            painter.end()

            # ハイライトされたROIをシーンに追加
            self.highlightedROIPixmapItem = QGraphicsPixmapItem(pixmap)
            self.scene.addItem(self.highlightedROIPixmapItem)

            # 透明度の設定
            opacityEffect = QGraphicsOpacityEffect()
            opacityEffect.setOpacity(self.current_opacity)
            self.highlightedROIPixmapItem.setGraphicsEffect(opacityEffect)
            
    # 選択したROIの形状およびtraceに関するパラメータを表示
    def displayROIstat(self, selected_row):
        for label_key in (["npix", "radius", "aspect_ratio", "compact", "footprint", "skew", "std"]):
            text = f"{label_key}: {self.dict_Fall_stat[selected_row][label_key][0][0]:.3f} "
            self.dict_label[f"ROI_{label_key}"].setText(text)

    def adjustContrast(self, channel, type_, value):
        # コントラスト値を更新
        self.contrastValues[channel][type_] = value
        # 画像表示を更新
        self.updateImageDisplay()
        
    def clearImageItems(self):
        for item in self.scene.items():
            if hasattr(item, 'is_image_item') and item.is_image_item:
                self.scene.removeItem(item)

    def updateImageDisplay(self):
        # 画像がチェックされていない場合は表示しない
        if not self.channel_visibility["Green"] and not self.channel_visibility["Red"]:
            self.clearImageItems()
            return

        # 画像をマージするための空の配列を作成
        merged_image = np.zeros((self.image_stacks["Green"].shape[1], self.image_stacks["Green"].shape[2], 3), dtype=np.uint8)

        # Greenチャンネルの画像を追加
        if self.channel_visibility["Green"]:
            green_image = self.adjustImageForDisplay(self.image_stacks["Green"][self.current_slice], "Green")
            merged_image[:, :, 1] = green_image

        # Redチャンネルの画像を追加, 読み込んでいない場合はスキップ
        if (self.channel_visibility["Red"]) and not (self.image_stacks["Red"][0] is None):
            red_image = self.adjustImageForDisplay(self.image_stacks["Red"][self.current_slice], "Red")
            merged_image[:, :, 0] = red_image

        # QImageオブジェクトを作成して表示
        qimage = QImage(merged_image.data, merged_image.shape[1], merged_image.shape[0], QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        self.clearImageItems()  # 以前の画像アイテムのみをクリア
        pixmap_item = self.scene.addPixmap(pixmap)
        pixmap_item.is_image_item = True  # 画像アイテムであることをタグ付け

        # ROIの画像を重ねて表示
        if self.roiPixmapItem:
            self.roiPixmapItem.setVisible(self.showAllROIs)
            self.roiPixmapItem.setZValue(1)

        # ハイライトされたROIを再描画（現在選択されているROIがある場合）
        if self.currently_selected_row is not None:
            self.highlightROI(self.currently_selected_row)

    def adjustImageForDisplay(self, image, channel):
        # 画像のコントラストを調整し、適切な形式で返す
        min_val = self.contrastValues[channel]['min']
        max_val = self.contrastValues[channel]['max']
        adjusted_image = np.clip(image, min_val, max_val)
        adjusted_image = np.interp(adjusted_image, (min_val, max_val), (0, 255))
        return adjusted_image.astype(np.uint8)
        
    def toggleChannelVisibility(self, channel, state):
        self.channel_visibility[channel] = state == Qt.Checked
        self.updateImageDisplay()  # このメソッド内でROIの表示状態も適切に更新する
        
    def toggleAllROIsVisibility(self, state):
        self.showAllROIs = state == Qt.Checked
        if self.roiPixmapItem:
            self.roiPixmapItem.setVisible(self.showAllROIs)

    def changeZSlice(self, value):
        self.current_slice = value
        # 両方のチャンネルに対して画像を更新
        for channel in self.image_stacks.keys():
            if self.channel_visibility[channel]:  # チャンネルが表示状態の場合のみ更新
                current_image = self.image_stacks[channel][self.current_slice]
                self.updateImageDisplay(channel, current_image)
                
    def changeAllROIOpacity(self, value):
        self.current_opacity_allroi = value
        id_showROI = self.dict_showROI[self.dict_buttongroup["roicheck"].checkedId()]
        self.createROIPixmap(id_showROI)
        
    def changeSelectedROIOpacity(self, value):
        self.current_opacity = value / 255.0  # 0から1の範囲に正規化
        if hasattr(self, 'highlightedROIPixmapItem') and self.highlightedROIPixmapItem:
            # QGraphicsOpacityEffectを使用して透明度を設定
            opacityEffect = QGraphicsOpacityEffect()
            opacityEffect.setOpacity(self.current_opacity)
            self.highlightedROIPixmapItem.setGraphicsEffect(opacityEffect)
            
    """
    Trace Plot, Axis Function
    """
    # 上段のプロット
    def plotTraceOnTopAxis(self, selected_row):
        self.dict_ax["top"].clear()  # 既存のプロットをクリア
        # プロット
        for trace, color, label in zip((self.Fall_F, self.Fall_Fneu, self.Fall_spks), ("cyan", "red", "gray"), ("raw fluor", "neuropil", "deconv")):
            # 時間スケールを秒に変換
            trace_plot = trace[selected_row]
            self.plot_x = np.arange(len(trace_plot)) / self.dict_Fall_ops["fs"][0][0]
            # 拡大あり
            self.dict_ax["top"].plot(self.plot_x, trace_plot, color=color, label=label, linewidth=0.5)
            
        # eventfileとの相関係数
        if self.load_eventfile:
            corrcoef = np.corrcoef(self.eventfile, self.Fall_F[selected_row])[0][1]
        else:
            corrcoef = 0
        self.dict_ax["top"].set_title(f"cell ID {selected_row}, corrcoef with Event = {corrcoef:.3f}")
        self.dict_ax["top"].set_xlabel(f"sec")
        
        # EventFileのプロット
        if (self.load_eventfile):
            self.plotEventFileAlignedTrace(selected_row)
            if self.dict_checkbox["plotEventFileTrace"].isChecked(): # F traceに重ねるか
                self.plotEventFile(self.dict_ax["top"], scale=np.max(self.Fall_F[selected_row]))
    
    # 中段のプロット
    def plotTraceOnMiddleAxis(self, selected_row):
        self.dict_ax["middle"].clear()
        for trace, color, label in zip((self.Fall_F, self.Fall_Fneu, self.Fall_spks), ("cyan", "red", "gray"), ("raw fluor", "neuropil", "deconv")):
            # 時間スケールを秒に変換
            trace_plot = trace[selected_row]
            self.plot_x = np.arange(len(trace_plot)) / self.dict_Fall_ops["fs"][0][0]
            self.dict_ax["middle"].plot(self.plot_x, trace_plot, color=color, label=label, linewidth=0.5)
            
        # EventFileのプロット
        if (self.load_eventfile):
            if self.dict_checkbox["plotEventFileTrace"].isChecked(): # F traceに重ねるか
                self.plotEventFile(self.dict_ax["middle"], scale=np.max(self.Fall_F[selected_row]))

    def updateSelectedCellPlot(self, selected_row):
        # 選択された細胞のデータをプロット
        if selected_row is not None:
            # 選択された細胞のデータを取得
            self.plotTraceOnTopAxis(selected_row)
            self.plotTraceOnMiddleAxis(selected_row)
                
#         self.figure.tight_layout()
        self.canvas.draw()
        
        if self.xlim:
            self.dict_ax["top"].set_xlim(self.xlim)  # 保存された表示範囲を適用
            self.drawPlotRangeRectangle()
            
        self.canvas.draw_idle()
        
    def plotMeanTrace(self):
        self.dict_ax["bottom"].clear()  # 既存のプロットをクリア
        # 全細胞の平均データをプロット
        for trace, color, label in zip((self.Fall_F, self.Fall_Fneu, self.Fall_spks), ("cyan", "red", "gray"), ("raw fluor", "neuropil", "deconv")):
            mean_data = np.mean(trace, axis=0)
            self.dict_ax["bottom"].plot(np.arange(len(mean_data)) / self.dict_Fall_ops["fs"][0][0], mean_data, color=color, label=label, linewidth=0.5)
        self.dict_ax["bottom"].set_title("average trace")
        self.Fall_F_mean = np.mean(self.Fall_F, axis=0)
        self.canvas.draw()  # キャンバスを更新してプロットを表示 
        
    # EventFileのプロット
    def plotEventFile(self, ax, scale=1):
        eventfile = self.eventfile.copy()
        eventfile *= scale # スケーリング
        ax.plot(self.plot_x, eventfile, color="green", linewidth=0.5)
        self.canvas.draw()
        
    # EventFileにalignしたtraceをプロット
    def plotEventFileAlignedTrace(self, selected_row):
        self.dict_ax["bottom"].clear()
        
        eventfile = self.eventfile.copy()
        trace_plot = self.Fall_F[selected_row]
        
        eventOnset_idxs = np.where(np.diff(eventfile, prepend=0)==1)[0]
        # eventOnsetから前後を抽出 エラーが発生した場合が省く
        list_trace_event = []
        range_event = self.dict_entry["EventFileAlignedTrace"].text().replace("(", "").replace(")", "").replace(" ","")
        range_event_pre, range_event_post = int(range_event.split(",")[0]), int(range_event.split(",")[1])
        plot_x = np.arange(-range_event_pre, range_event_post) / self.dict_Fall_ops["fs"][0][0]
        for eventOnset_idx in eventOnset_idxs:
            if not (eventOnset_idx-range_event_pre < 0) and not (eventOnset_idx+range_event_post >= len(trace_plot)): # 範囲外はスキップ
                list_trace_event.append(trace_plot[eventOnset_idx-range_event_pre:eventOnset_idx+range_event_post])
            
        mean_trace_event = np.mean(list_trace_event, axis=0)
        self.dict_ax["bottom"].plot(plot_x, mean_trace_event, color="red", linewidth=0.5, zorder=2)
        plot_event = eventfile[eventOnset_idxs[0]-range_event_pre:eventOnset_idxs[0]+range_event_post]
        plot_event *= np.max(mean_trace_event) # スケーリング
        self.dict_ax["bottom"].plot(plot_x, plot_event, color="green", linewidth=0.5, zorder=3)
        for trace_event in list_trace_event:
            self.dict_ax["bottom"].plot(plot_x, trace_event, linewidth=0.2, zorder=1)
        self.dict_ax["bottom"].set_title("event aligned trace")
        self.dict_ax["bottom"].set_xlabel("sec")
        self.canvas.draw()
        
    # 上段でプロットしている範囲を中段にプロットする
    def drawPlotRangeRectangle(self):
        xmin, xmax = self.dict_ax["top"].get_xlim()
        ymin, ymax = self.dict_ax["top"].get_ylim()
        
        self.plotTraceOnMiddleAxis(self.currently_selected_row)
        x_, y_ = (xmin, xmin, xmax, xmax, xmin), (ymin, ymax, ymax, ymin, ymin)
        self.dict_ax["middle"].plot(x_, y_, color="purple", linewidth=0.5)
        
    # plot範囲の拡大、縮小
    def on_scroll(self, event):
        # 上段でのみスクロール可
        if event.inaxes == self.dict_ax["top"]:
            # スクロールイベントでズーム処理
            ax = (self.dict_ax["top"],)  # 軸をタプルで取得

            # ズームレベルの制限を追加
            min_zoom_level = int(self.dict_entry["minPlotRange"].text())  # 最小ズームレベル（最大縮小範囲）
            max_zoom_level = self.Fall_F.shape[1] / self.dict_Fall_ops["fs"][0][0] * 1.05  # 最大ズームレベル（最大拡大範囲）
            for a in ax:
                xlim = a.get_xlim()
                xmouse = event.xdata  # マウスのx位置を取得
                if xmouse is None:  # マウスがプロット外にある場合は無視
                    return
                scale_factor = 0.5 if event.button == 'up' else 2.0

                xleft = xmouse - (xmouse - xlim[0]) * scale_factor
                xright = xmouse + (xlim[1] - xmouse) * scale_factor

                # 新しい表示範囲の幅
                new_width = xright - xleft
                if new_width < min_zoom_level:
                    new_width = min_zoom_level
                elif new_width > max_zoom_level:
                    new_width = max_zoom_level

                # 新しい表示範囲の中心を元のマウスの位置に合わせる
                center = (xright + xleft) / 2
                xleft = center - new_width / 2
                xright = center + new_width / 2

                # 元の表示範囲と比較して調整
                original_width = xlim[1] - xlim[0]
                if new_width == min_zoom_level and original_width <= min_zoom_level:
                    # 既に最小ズームレベルにある場合は何もしない
                    return
                elif new_width == max_zoom_level and original_width >= max_zoom_level:
                    # 既に最大ズームレベルにある場合は何もしない
                    return

                # 新しい表示範囲を設定
                a.set_xlim(xleft, xright)

            self.xlim = self.dict_ax["top"].get_xlim()  # ズーム操作完了時に表示範囲を保存
            self.drawPlotRangeRectangle()
            self.canvas.draw_idle()
        
    # plot範囲の移動
    def on_press(self, event):
        if event.button == 1:  # 左クリックの場合
            # 上段, ドラッグで移動
            if event.inaxes == self.dict_ax["top"]:
                self.drag_start_x = event.xdata
                self.is_dragging = True
            # 中段, クリックで移動
            elif event.inaxes == self.dict_ax["middle"]:
                if self.xlim and event.xdata is not None:
                    # 画面座標からデータ座標へ変換
                    x, y = event.inaxes.transData.inverted().transform((event.x, event.y))
                    xlim_min, xlim_max = self.xlim
                    xlim_range = (xlim_max - xlim_min)
                    # データ範囲外にならないように調整
                    if x < xlim_range / 2:
                        x = xlim_range / 2
                    elif x > (self.Fall_F.shape[1] / self.dict_Fall_ops["fs"][0][0]) - (xlim_range / 2):
                        x = (self.Fall_F.shape[1] / self.dict_Fall_ops["fs"][0][0]) - (xlim_range / 2)
                    xlim_center = x
                    # 新しい表示範囲を計算
                    new_xlim_min = max(xlim_center - xlim_range / 2, 0)  
                    new_xlim_max = min(xlim_center + xlim_range / 2, self.Fall_F.shape[1] / self.dict_Fall_ops["fs"][0][0])

                    # topの表示範囲を更新
                    self.dict_ax["top"].set_xlim(new_xlim_min, new_xlim_max)
                    self.xlim = new_xlim_min, new_xlim_max  # 表示範囲を保存
                    self.drawPlotRangeRectangle()
                    self.canvas.draw_idle()
    def on_motion(self, event):
        if self.is_dragging and event.xdata:
            dx = event.xdata - self.drag_start_x  # ドラッグによるx方向の移動量
            dx *= 1.5 # スケーリング
            for ax in (self.dict_ax["top"],):
                xlim = ax.get_xlim()
                ax.set_xlim(xlim[0] - dx, xlim[1] - dx)
            self.drawPlotRangeRectangle()
            self.canvas.draw_idle()
    def on_release(self, event):
        self.is_dragging = False
        if self.dict_ax["top"].get_xlim():
            self.xlim = self.dict_ax["top"].get_xlim()  # ドラッグ操作完了時に表示範囲を保存
        self.drawPlotRangeRectangle()
            
    """
    Button Function
    """
    # 全てのthresholdsのmin~maxに収まるROI以外をNot cellとする
    def filterROI(self):
        reply = QMessageBox.question(self, 'Filter Cells', 'Filter ROIs?', QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            # noiseとフィルタリングした行にチェックをつけるか
            reply = QMessageBox.question(self, 'Check Cells', 'Check cells determined "Not cell"?', QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
            if reply == QMessageBox.Yes:
                check = True
            else:
                check = False
            # threshold
            lists_roifilter = []
            for i, param in enumerate(["npix", "radius", "aspect_ratio", "compact", "skew", "std"]):
                threshold_range = self.dict_entry[f"threshold_{param}"].text().replace("(", "").replace(")", "").replace(" ","")
                try:
                    threshold_min, threshold_max = float(threshold_range.split(",")[0]), float(threshold_range.split(",")[1])
                    list_roi_param = [self.dict_Fall_stat[row][param] for row in self.dict_Fall_stat.keys()]
                    list_roifilter = [(roi_param >= threshold_min) and (roi_param <= threshold_max) for roi_param in list_roi_param]
                    lists_roifilter.append(list_roifilter)
                except ValueError:
                    pass

            # Cellと判定されたROI
            list_roi_cell = [all(values) for values in zip(*lists_roifilter)]
            
            for cellid, roi_cell in enumerate(list_roi_cell):
                radioButton = self.tableWidget.cellWidget(cellid, 3) # QRadioButtonを取得
                checkBox = self.tableWidget.item(cellid, 4)
                if not roi_cell:
                    radioButton.setChecked(True)  # 選択状態を更新
                    if check: # チェックつけるか
                        checkBox.setCheckState(Qt.Checked)
                        
    # すべてのROIのクラスをそろえる Astro, Neuron, Noise
    def setAllROIClass(self, celllabel):
        if celllabel not in self.dict_celltypes:
            return
        reply = QMessageBox.question(self, 'Set All Cells', f'Set all ROIs as {celllabel}?', QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            for row in range(self.tableWidget.rowCount()):
                checkBox = self.tableWidget.item(row, self.dict_celltypes["Check"])
                isChecked = checkBox.checkState() == Qt.Checked
                if not isChecked:
                    radioButton = self.tableWidget.cellWidget(row, self.dict_celltypes[celllabel])
                    radioButton.setChecked(True)
    
    # すべてのROIのチェックをつける
    def setAllROICheckBox(self):
        reply = QMessageBox.question(self, 'Check All Cells', f'Check all ROIs?', QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            for row in range(self.tableWidget.rowCount()):
                checkBox = self.tableWidget.item(row, self.dict_celltypes["Check"])
                checkBox.setCheckState(Qt.Checked)
    # すべてのROIのチェックを外す
    def clearAllROICheckBox(self):
        reply = QMessageBox.question(self, 'Uncheck All Cells', f'Uncheck all ROIs?', QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            for row in range(self.tableWidget.rowCount()):
                checkBox = self.tableWidget.item(row, self.dict_celltypes["Check"])
                checkBox.setCheckState(Qt.Unchecked)
            
    # ROICheckファイルの保存
    def saveROICheck(self):
        options = QFileDialog.Options()
        path_Fall = self.dict_entry["path_fall"].text().split("/")[-1].replace('Fall_', '')
        path_roicheck_init = f"{self.dir_project}/ROIcheck_{path_Fall}"
        path_roicheck, _ = QFileDialog.getSaveFileName(self, "Open File", path_roicheck_init, "mat Files (*.mat);;All Files (*)", options=options)
        if path_roicheck:
            today = datetime.datetime.today().strftime('%Y-%m-%d')
            today = today.split("-")[0][2:] + today.split("-")[1] + today.split("-")[2]

            rows_selected_neuron = []
            rows_selected_astro = []
            rows_selected_noise = []
            checkStates = []
            trackingStates = []
            memoContents = []

            for row in range(self.tableWidget.rowCount()):
                for celltype in self.dict_celltypes:
                    if celltype not in ["Check", "Tracking", "Memo"] and self.buttonGroups[row].checkedButton().text() == celltype:
                        if celltype.startswith("Neuron"):
                            rows_selected_neuron.append([row])
                        elif celltype.startswith("Astro"):
                            rows_selected_astro.append([row])
                        elif celltype == "Not Cell":
                            rows_selected_noise.append([row])
                        break
                checkItem = self.tableWidget.item(row, self.dict_celltypes["Check"])
                checkStates.append([checkItem.checkState() == Qt.Checked])
                trackingItem = self.tableWidget.item(row, self.dict_celltypes["Tracking"])
                trackingStates.append([trackingItem.checkState() == Qt.Checked])
                memoItem = self.tableWidget.item(row, self.dict_celltypes["Memo"])
                memoText = memoItem.text() if memoItem else ""
                memoContents.append([memoText])

            dict_threshold_roi = {param: self.dict_entry[f"threshold_{param}"].text() for param in ["npix", "radius", "aspect_ratio", "compact", "skew", "std"]}

            mat_roicheck = {
                "manualROIcheck": {
                    "rows_selected_neuron": np.array(rows_selected_neuron),
                    "rows_selected_astro" : np.array(rows_selected_astro),
                    "rows_selected_noise" : np.array(rows_selected_noise),
                    "update"              : today,
                    "Check"               : np.array(checkStates),
                    "Tracking"            : np.array(trackingStates),
                    "Memo"                : np.array(memoContents).astype("object"),
                    "threshold_roi"       : dict_threshold_roi,
                }
            }

            savemat(path_roicheck, mat_roicheck)
            print("ROICheck file saved!")
        
    # ROICheckファイルの読み込み
    def loadROICheck(self):
        options = QFileDialog.Options()
        path_roicheck, _ = QFileDialog.getOpenFileName(self, "Open File", "", "mat Files (*.mat);;All Files (*)", options=options)
        if path_roicheck:
            mat_roicheck = loadmat(path_roicheck)
            mat_roicheck_dtype = list(mat_roicheck["manualROIcheck"][0].dtype.fields)
            dict_roicheck = dict(zip(mat_roicheck_dtype, list(mat_roicheck["manualROIcheck"][0][0])))
                        
            for row in range(self.tableWidget.rowCount()):
                for celltype in ["rows_selected_neuron", "rows_selected_astro", "rows_selected_noise"]:
                    if row in dict_roicheck[celltype].flatten():
                        if celltype == "rows_selected_neuron":
                            radioButton = self.tableWidget.cellWidget(row, self.dict_celltypes["Neuron"])
                        elif celltype == "rows_selected_astro":
                            radioButton = self.tableWidget.cellWidget(row, self.dict_celltypes["Astrocyte"])
                        elif celltype == "rows_selected_noise":
                            radioButton = self.tableWidget.cellWidget(row, self.dict_celltypes["Not Cell"])
                        radioButton.setChecked(True)

                for celltype in ["Check", "Tracking"]:
                    checkBox = self.tableWidget.item(row, self.dict_celltypes[celltype])
                    if dict_roicheck[celltype][row][0]:
                        checkBox.setCheckState(Qt.Checked)
                    else:
                        checkBox.setCheckState(Qt.Unchecked)

                memoItem = self.tableWidget.item(row, self.dict_celltypes["Memo"])
                try:
                    memoItem.setText(str(dict_roicheck["Memo"][row][0][0]))
                except IndexError:
                    pass
                        
            # ROI thresholdの更新
            dict_roicheck_threshold_roi_dtype = list(dict_roicheck["threshold_roi"][0].dtype.fields)
            dict_threshold_roi = dict(zip(dict_roicheck_threshold_roi_dtype, list(dict_roicheck["threshold_roi"][0][0])))
            for param in ["npix", "radius", "aspect_ratio", "compact", "skew", "std"]:
                text = dict_threshold_roi[param][0]
                self.dict_entry[f"threshold_{param}"].setText(text)

            print("ROICheck file loaded!")
            
    # ROIMatch csvとfixのROICheck matを読み込んでtableを更新
    def loadROIMatch(self):
        options = QFileDialog.Options()
        path_roimatch, _ = QFileDialog.getOpenFileName(self, "Open File", "", "csv Files (*.csv);;All Files (*)", options=options)
        path_roicheck, _ = QFileDialog.getOpenFileName(self, "Open File", "", "mat Files (*.mat);;All Files (*)", options=options)
        if (path_roimatch) and (path_roicheck):
            mat_roicheck = loadmat(path_roicheck)
            mat_roicheck_dtype = list(mat_roicheck["manualROIcheck"][0].dtype.fields)
            dict_roicheck = dict(zip(mat_roicheck_dtype, list(mat_roicheck["manualROIcheck"][0][0])))
            
            df_roimatch = pd.read_csv(path_roimatch)

            for row in range(self.tableWidget.rowCount()):
                # Match Cell ID が-1でないもののみ更新
                if int(df_roimatch.iloc[row, 1]) >= 0: 
                    for celltype in ["rows_selected_neuron", "rows_selected_astro", "rows_selected_noise"]:
                        if row in dict_roicheck[celltype].flatten():
                            if celltype == "rows_selected_neuron":
                                radioButton = self.tableWidget.cellWidget(row, self.dict_celltypes["Neuron"])
                            elif celltype == "rows_selected_astro":
                                radioButton = self.tableWidget.cellWidget(row, self.dict_celltypes["Astrocyte"])
                            elif celltype == "rows_selected_noise":
                                radioButton = self.tableWidget.cellWidget(row, self.dict_celltypes["Not Cell"])
                            radioButton.setChecked(True)

                    for celltype in ["Check", "Tracking"]:
                        checkBox = self.tableWidget.item(row, self.dict_celltypes[celltype])
                        if dict_roicheck[celltype][row][0]:
                            checkBox.setCheckState(Qt.Checked)
                        else:
                            checkBox.setCheckState(Qt.Unchecked)

                    memoItem = self.tableWidget.item(row, self.dict_celltypes["Memo"])
                    try:
                        memoItem.setText(str(dict_roicheck["Memo"][row][0][0]))
                    except IndexError:
                        pass
        
    # EventFileの読み込み
    def loadEventFile(self):
        options = QFileDialog.Options()
        # npyを連結できるように複数ファイルを選択可能
        paths_eventfile, _ = QFileDialog.getOpenFileNames(self, "Open File", "", "npy Files (*.npy);;All Files (*)", options=options)
        if paths_eventfile:
            # 複数読み込んで連結
            self.eventfile = np.concatenate([np.load(path_eventfile) for path_eventfile in paths_eventfile])
            self.load_eventfile = True
            self.updateSelectedCellPlot(self.currently_selected_row)
            self.canvas.draw()
            self.canvas.draw_idle()
            print("EventFile loaded !")
    def clearEventFile(self):
        self.eventfile = None
        self.load_eventfile = False
        self.updateSelectedCellPlot(self.currently_selected_row)
        self.plotMeanTrace()
        self.canvas.draw()
        self.canvas.draw_idle()
            
    # Help
    def showHelp(self):
        # ヘルプダイアログを表示
        title = "Suite2pROICheckGUI Help"
        contents = "How to use<br>\
                    <br>\
                    * With the right table selected<br>\
                    Z : Check 'Astrocyte'<br>\
                    X : Check 'Neuron'<br>\
                    C : Check 'Not Cell'<br>\
                    V : Check 'Check'<br>\
                    B : Check 'Tracking'<br>\
                    G : Check 'Show Green'<br>\
                    ↑ : Select one row up<br>\
                    ↓ : Select one row down<br>\
                    U : Select one row up (with same cell type)<br>\
                    J : Select one row down (with same cell type)<br>\
                    I : Select one row up (skip rows with checked 'Check' column)<br>\
                    K : Select one row down (skip rows with checked 'Check' column)<br>\
                    O : Select one row up (skip rows without checked 'Check' column)<br>\
                    L : Select one row down (skip rows without checked 'Check' column)<br>\
                    <br>\
                    * With 'Move ROI Mask Image' checked<br>\
                    W : Move ROI Mask Image up<br>\
                    A : Move ROI Mask Image right<br>\
                    S : Move ROI Mask Image down<br>\
                    D : Move ROI Mask Image left<br>\
                    "

        style = "font-size: 12pt;"
        dialog = HelpDialog(self, style, title, contents)
        dialog.show()

    def exitApp(self):
        self.close()
        
        
class HelpDialog(QDialog):
    def __init__(self, parent=None, style=None, title="", contents="", geometry=(100, 100, 500, 400)):
        super(HelpDialog, self).__init__(parent)
        self.setWindowTitle(title)
        self.setGeometry(*geometry)  # アンパック演算子を使用して、geometryを直接適用
        
        # ヘルプの内容を表示するテキスト
        helpText = QLabel(contents, self)
        helpText.setWordWrap(True)  # テキストの折り返しを有効にする
        if style:
            helpText.setStyleSheet(style)  # スタイルシートを適用
        
        # レイアウト設定
        layout = QVBoxLayout()
        layout.addWidget(helpText)
        self.setLayout(layout)

if __name__ == "__main__":
    # フォント揃え
    restored_font = QFont()
    restored_font.fromString("MS UI Gothic,9,-1,5,50,0,0,0,0,0")
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    QApplication.setFont(restored_font)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

Files Loading...
GUI Initializing...


SystemExit: 0